# Homework 3
### Optimization with Stochastic Gradient Descent

In [1]:
import numpy as np
import matplotlib.pylab as plt

from math import ceil

In [2]:
def SGD(l, grad_l, w0, data, batch_size, n_epochs, alpha=0.01):
    """
        Arguments:
            l: the function l(w; D) we want to optimize. It is supposed to be a Python function, not an array.
            grad_l: the gradient of l(w; D). It is supposed to be a Python function, not an array.
            w0: an n-dimensional array which represents the initial iterate. By default, it should be randomly sampled.
            data: a tuple (x, y) that contains the two arrays x and y, where x is the input data, y is the output data.
            batch_size: an integer. The dimension of each batch. Should be a divisor of the number of data.
            n_epochs: an integer. The number of epochs you want to repeat the iterations.
        Returns:
            w: an array that contains the value of w_k FOR EACH iterate w_k (not only the latter).
            f_val: an array that contains the value of l(w_k; data) FOR EACH iterate w_k ONLY after each epoch.
            grads: an array that contains the value of grad_l(w_k; data) FOR EACH iterate w_k ONLY after each epoch.
            err: an array the contains the value of ||grad_l(w_k; data)||_2 FOR EACH iterate w_k ONLY after each epoch.
    """
    X, Y = data
    d, N = X.shape

    # number of epochs * iterations per epoch
    tot_iterations = ceil(n_epochs * ceil(N / batch_size))

    w = np.zeros(tot_iterations + 1, dtype=object)
    f_val = np.zeros(n_epochs, dtype=object)
    grads = np.zeros(n_epochs, dtype=object)
    err = np.zeros(n_epochs, dtype=object)

    w[0] = w0
    k = 0

    for epoch in range(n_epochs):
        print(f"--> Epoch {epoch}")

        # Shuffle the dataset
        rng = np.random.default_rng()
        D = np.concatenate((Y, X.T), axis=1)
        rng.shuffle(D)
        X = D[:, 1:].T
        Y = D[:, 0].reshape(-1, 1)
        
        batches_num = ceil(N / batch_size)
        for batch in range(batches_num):
            # print(f"\tBatch {batch}")
            Mx = X[:, batch*batch_size:(batch+1)*batch_size]
            My = Y[batch*batch_size:(batch+1)*batch_size]
            M = (Mx, My)
            # Gradient descent step
            k = (epoch * batches_num) + batch
            w[k+1] = w[k] + alpha * grad_l(w[k], M) 

        f_val[epoch] = l(w[k+1], data)
        grads[epoch] = grad_l(w[k+1], data)
        
        err[epoch] = np.linalg.norm(grads[epoch])
        print(err[epoch])
    
    return w, f_val, grads, err

In [3]:

# Load MINST dataset
import pandas as pd
df = pd.read_csv("../data/data.csv")
data_raw = np.array(df)
X_raw = data_raw[:, 1:]
y_raw = data_raw[:, 0]
X_raw.shape, y_raw.shape

((42000, 784), (42000,))

In [94]:
digit_list = [0, 9]

mask = [y_i in digit_list for y_i in y_raw]
y = y_raw[mask]
X = X_raw[mask, :]
X.shape, y.shape

((8320, 784), (8320,))

In [95]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, shuffle=False)
X_train = X_train.T
X_test = X_test.T
y_train = y_train.reshape((-1, 1))
y_test = y_test.reshape((-1, 1))
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((784, 6656), (784, 1664), (6656, 1), (1664, 1))

In [96]:
def sigmoid(x):
    return 1./(1.+np.exp(-x))

def der_sigmoid(x):
    return sigmoid(x) * (1 - sigmoid(x))

In [97]:
def l(w, D):
    X, y = D
    d, N = X.shape
    return np.sum(np.power(np.linalg.norm(sigmoid(X.T @ w) - y, axis=1), 2))/N

def grad_l(w, D):
    X, y = D
    d, N = X.shape
    der = der_sigmoid(X.T @ w)
    res = der * X.T * (sigmoid(X.T @ w) - y)
    temp = (2*np.sum(res, axis=0)/N).reshape((-1,1))
    return temp

def grad_l2(w, D):
    X, y = D
    d, N = X.shape
    def grad_step(arg):
        x_i, y_i = (arg[1:], arg[:1])
        res_step = der_sigmoid(x_i.T @ w) * x_i.T * (sigmoid(x_i.T @ w) - y_i)
        return res_step
    full_D = np.concatenate((y.T, X), axis=0)
    tmp = np.apply_along_axis(grad_step, 0, full_D)
    res = np.sum(tmp, axis=1) / N
    return res.reshape((-1,1))



In [98]:
# init weights
rng = np.random.default_rng()
w0 = rng.normal(size=(X_train.shape[0] + 1, 1))
X_hat = np.concatenate((np.ones((1, X_train.shape[1])), X_train), axis=0)

In [ ]:
n_epochs = 20
batch_size = 1000
w, f_val, grads, err = SGD(l, grad_l2, w0, (X_hat, y_train), batch_size, n_epochs, alpha=0.5)

In [ ]:
plt.plot([x for x in range(1, n_epochs + 1)], err)

In [ ]:
X_test_hat = np.concatenate((np.ones((1, X_test.shape[1])), X_test), axis=0)
predictions = sigmoid(X_test_hat.T @ w[-1])
y_predicted = np.apply_along_axis(lambda x: 9 if x < 0.5 else 0, 1, predictions).reshape((-1,1))

In [ ]:
def compute_accuracy(y_true, y_guess):
    return (len(y_true) - np.count_nonzero(y_true - y_guess)) / len(y_true) * 100

In [ ]:
compute_accuracy(y_predicted, y_test)

In [111]:
def GD_log_reg(f, grad_f, w0, D, tolf, tolx, kmax, alpha=0.1):
    # As output, we want:
    # x -> The stationary point
    # f_val -> A vector containing the values of f during the iterations
    # err_val -> A vector containing the values of ||grad_f(x_k)||

    f_val = [f(w0, D)]
    err_val = [grad_f(w0, D)]
    
    # Init iteration
    k = 0
    x = [w0]
    
    rel_tol_cond = True
    rel_inp_cond = True
    max_it_cond = True
    
    while rel_tol_cond and rel_inp_cond and max_it_cond:
        x.append(x[k] - alpha * grad_f(x[k], D))
        f_val.append(f(x[k+1], D))
        err_val.append(np.linalg.norm(grad_f(x[k], D)))
        
        k += 1
        if(k % 100 == 0): print(f"iteration: {k}")

        rel_tol_cond = np.linalg.norm(grad_f(x[k-1], D)) > tolf * np.linalg.norm(grad_f(w0, D))
        # We skip the first iteration
        rel_inp_cond = True if k == 0 else np.linalg.norm(x[k] - x[k-1]) > tolx
        max_it_cond = k < kmax

    return x[k], f_val, err_val

In [112]:
res_gd = GD_log_reg(l, grad_l2, w0, (X_hat, y_train), 0., 0., 500, alpha=0.4)
w_k, f_val, err_val = res_gd

/var/folders/m9/2qy1g_cd5mjcg_cqty8ydzq00000gn/T/ipykernel_1211/1431357966.py:2: RuntimeWarning: overflow encountered in exp
  return 1./(1.+np.exp(-x))


iteration: 100
iteration: 200
iteration: 300
iteration: 400
iteration: 500


In [113]:
X_test_hat = np.concatenate((np.ones((1, X_test.shape[1])), X_test), axis=0)
predictions = sigmoid(X_test_hat.T @ w_k)
y_predicted = np.apply_along_axis(lambda x: 9 if x < 0.5 else 0, 1, predictions).reshape((-1,1))
compute_accuracy(y_test, y_predicted)

50.54086538461539

In [110]:
print(f"class 0: {y_predicted[y_predicted == 0].shape[0]}\nclass 9: {y_predicted[y_predicted == 9].shape[0]}")

class 0: 1637
class 9: 27
